In [1]:
!pip install --quiet --upgrade pydantic-ai neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.4/496.4 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.3/325.3 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**Pydantic AI** is a Python framework for building production-ready AI agents. Leveraging Pydantic's powerful validation and serialization capabilities, it provides type-safe tool definitions, structured outputs, dependency injection, and seamless integration with multiple LLM providers.

In this notebook, we'll build a company research agent that queries a Neo4j graph database.

In [2]:
import os
import json
import base64

import requests

from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerStreamableHTTP

from neo4j import AsyncGraphDatabase

Pydantic AI is model-agnostic and supports virtually every major provider including OpenAI, Anthropic, Gemini, Amazon Bedrock, Google Vertex AI, and Azure AI Foundry. Here we will be using OpenAI.

In [3]:
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key")

OpenAI API key··········


## Neo4j MCP Server
We'll start by using the official [Neo4j MCP Server](https://github.com/neo4j/mcp) to extend the agent with Neo4j tools. This MCP server provides the agent with capabilities to read the graph schema and execute Cypher queries, enabling it to fetch and analyze data directly from the database.

The following code installs the latest version on Google Colab and similar Linux-based systems. For other operating systems, please consult the [official installation documentation](https://neo4j.com/docs/mcp/current/installation/).

In [4]:
# Get latest release info from GitHub API
release = requests.get("https://api.github.com/repos/neo4j/mcp/releases/latest").json()
version = release["tag_name"]
print(f"Latest version: {version}")

# Download the latest Linux binary
!wget -q https://github.com/neo4j/mcp/releases/download/{version}/neo4j-mcp_Linux_x86_64.tar.gz

# Extract
!tar -xzf neo4j-mcp_Linux_x86_64.tar.gz

# Make executable
!chmod +x neo4j-mcp

# Cleanup
!rm neo4j-mcp_Linux_x86_64.tar.gz

# Move
!mv neo4j-mcp /usr/local/bin/

# Verify installation
!neo4j-mcp -v

Latest version: v1.1.1
neo4j-mcp version: v1.1.1


For this example, we'll use the companies database from the Neo4j demo server, which contains organizations, people, investors, and news articles.

For HTTP transport, you only need to set the `NEO4J_URI` and optionally `NEO4J_DATABASE` (if connecting to a specific database).

In [5]:
os.environ["NEO4J_URI"] = "neo4j+s://demo.neo4jlabs.com"
os.environ["NEO4J_DATABASE"] = "companies"
os.environ["NEO4J_MCP_TRANSPORT"] = "http"

# Run the server in the background
import subprocess
subprocess.Popen(["neo4j-mcp"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

<Popen: returncode: None args: ['neo4j-mcp']>

Here we create an MCP server connection that connects to our Neo4j server endpoint. Credentials are passed via the `Authorization` header using Basic authentication. The HTTP transport listens on port 80 by default, so the MCP endpoint is available at `http://localhost:80/mcp`.

In [6]:
# Credentials are passed via bearer auth
os.environ["NEO4J_USERNAME"] = "companies"
os.environ["NEO4J_PASSWORD"] = "companies"

credentials = base64.b64encode(f"{os.environ["NEO4J_USERNAME"]}:{os.environ["NEO4J_PASSWORD"]}".encode()).decode()

mcp_server = MCPServerStreamableHTTP(
    'http://localhost:80/mcp',
    headers={
        'Authorization': f'Basic {credentials}'
    },
)

We can then pass the server directly as a toolset to an `Agent`, which automatically discovers and exposes all tools from the MCP server.

In [7]:
agent = Agent('openai:gpt-5', toolsets=[mcp_server])

Let's test it!

In [8]:
async with agent:
    result = await agent.run("How many people are in the database?")

    for msg in result.all_messages():
        for part in msg.parts:
            match part.part_kind:
                case "tool-call":
                    print(f"\n🔧 TOOL CALL: {part.tool_name}")
                    print(f"   └─ Args: {part.args}")

                case "tool-return":
                    print(f"   └─ Result: {part.content}")
                    print("-" * 50)

                case "text":
                    print(part.content, end="", flush=True)

    print(f"\n\n✅ Final: {result.output}")


🔧 TOOL CALL: get-schema
   └─ Args: {}
   └─ Result: [{'key': '_Bloom_Perspective_', 'value': {'type': 'node', 'properties': {'data': 'STRING', 'id': 'STRING', 'name': 'STRING', 'roles': 'LIST', 'version': 'STRING'}, 'relationships': {'_Bloom_HAS_SCENE_': {'direction': 'out', 'labels': ['_Bloom_Scene_']}}}}, {'key': 'HAS_CEO', 'value': {'type': 'relationship'}}, {'key': 'Organization', 'value': {'type': 'node', 'properties': {'diffbotId': 'STRING', 'id': 'STRING', 'isDissolved': 'BOOLEAN', 'isPublic': 'BOOLEAN', 'motto': 'STRING', 'name': 'STRING', 'nbrEmployees': 'INTEGER', 'revenue': 'FLOAT', 'summary': 'STRING'}, 'relationships': {'HAS_BOARD_MEMBER': {'direction': 'out', 'labels': ['Person']}, 'HAS_CATEGORY': {'direction': 'out', 'labels': ['IndustryCategory']}, 'HAS_CEO': {'direction': 'out', 'labels': ['Person']}, 'HAS_COMPETITOR': {'direction': 'out', 'labels': ['Organization']}, 'HAS_INVESTOR': {'direction': 'out', 'labels': ['Person', 'Organization']}, 'HAS_SUPPLIER': {'direct

## Custom Tools
Beyond using existing MCP servers, you can also implement your own custom tools and add them directly to the agent. This allows you to create specialized functionality tailored to your specific use case.

First, we need to define a Neo4j driver to establish a connection to the database and an `execute_cypher` helper method to run queries.

In [9]:
# Create async driver
driver = AsyncGraphDatabase.driver(os.environ["NEO4J_URI"], auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"]))


async def execute_cypher(cypher: str, params: dict) -> list[dict]:
    result = await driver.execute_query(cypher, database_=os.environ["NEO4J_DATABASE"], **params)
    return [record.data() for record in result.records]

The first custom tool uses a fulltext index to search for companies by name. With Pydantic AI, you can pass function directly to the agent.

In [10]:
async def find_companies(company_name: str) -> list[dict]:
    """List of Companies (company_id, name, summary) by fulltext search."""
    query = """
        CALL db.index.fulltext.queryNodes('entity', $search, {limit: 100})
        YIELD node as c, score WHERE c:Organization
        AND NOT EXISTS { (c)<-[:HAS_SUBSIDIARY]-() }
        RETURN c.id as company_id, c.name as name, c.summary as summary LIMIT 25
    """
    results = await execute_cypher(query, {"search": company_name})
    return results

The second tool retrieves investors for a given company, returning their IDs, names, and types (e.g., Organization or Person).

In [11]:
async def get_investments(company: str) -> list[dict]:
    """Returns the investments by a company by name. Returns list of investment ids, names and types."""
    results = await execute_cypher("""
        MATCH (o:Organization)-[:HAS_INVESTOR]->(i)
        WHERE o.name = $company
        RETURN i.id as id, i.name as name, head(labels(i)) as type
    """, {"company": company})
    return results

We can pass custom tools alongside the MCP server toolset by passing them to the `tools` parameter.

In [12]:
agent = Agent(
    "openai:gpt-5", toolsets=[mcp_server], tools=[find_companies, get_investments]
)


Let's test it!

In [14]:
async with agent:
    result = await agent.run("Which companies has Google invested in?")

    for msg in result.all_messages():
        for part in msg.parts:
            match part.part_kind:
                case "tool-call":
                    print(f"\n🔧 TOOL CALL: {part.tool_name}")
                    print(f"   └─ Args: {part.args}")

                case "tool-return":
                    print(f"   └─ Result: {part.content}")
                    print("-" * 50)

                case "text":
                    print(part.content, end="", flush=True)

    print(f"\n\n✅ Final: {result.output}")


🔧 TOOL CALL: get_investments
   └─ Args: {"company":"Google"}
   └─ Result: [{'id': 'ELsv5bECSOiWG_Uhf_txI2w', 'name': 'Ionic Security', 'type': 'Organization'}, {'id': 'EUkm62r-bMOidNtPjTkdVvg', 'name': 'Avere Systems', 'type': 'Organization'}, {'id': 'EX-RLztfkOFqTLoM6xIVnlg', 'name': 'FlexiDAO', 'type': 'Organization'}, {'id': 'EtqXbQ9LaMGq8om4dhYY0Fw', 'name': 'Cloudflare', 'type': 'Organization'}, {'id': 'EWIvDLNCSMCCBYUyz0oFPVQ', 'name': 'Trifacta', 'type': 'Organization'}]
--------------------------------------------------
Based on the current data I have, Google has invested in:
- Ionic Security
- Avere Systems
- FlexiDAO
- Cloudflare
- Trifacta

Would you like details on any of these investments (e.g., type, date, amount)?

✅ Final: Based on the current data I have, Google has invested in:
- Ionic Security
- Avere Systems
- FlexiDAO
- Cloudflare
- Trifacta

Would you like details on any of these investments (e.g., type, date, amount)?


## Summary

In this notebook, we built a company research agent using **Pydantic AI** with Neo4j:

1. **Pydantic AI Setup** — Configured a model-agnostic agent using OpenAI as the LLM provider, leveraging Pydantic's type-safe tool definitions and structured outputs

2. **Neo4j MCP Integration** — Connected to Neo4j's demo companies database using the official Neo4j MCP Server via HTTP transport, passing it directly to the Agent as a toolset that automatically discovers and exposes all available tools

3. **Custom Tools** — Created specialized Python functions (`find_companies`, `get_investments`) that query the graph database using fulltext search and relationship traversal

Pydantic AI makes it straightforward to integrate external MCP servers, define custom tools with automatic type inference, and build production-ready agents that can query and analyze graph data.